# 10.2 Sorting Algorithms

the standard implementation of sorting in most Python implementations runs in roughly $O(n*log(n))$ time, where n is the length of the list.

In most cases, the right thing to do is to use with Python

* 1  method **list.sort** : takes a list as its first argument and **modifies** that list,sorts the list (ascending sort),
    
    
* 2 function **sorted** : takes an iterable object (e.g., a list or a dictionary) as its first argument and returns a **new** sorted list, <b>does not mutate L</b>

In [ ]:
L=[1,34,5,7,90,2]
L.sort()
print('sorted L',L)
L.reverse()
print('reversed L',L)

In [ ]:
L =[1,34,5,7,90,2]
L1 = L[ : ]
# descending sorts
L1.sort(reverse = True)
print(L)
print(L1)

In [ ]:
L=[1,34,5,7,90,2]
# returns a new list with same elements as L
L1=sorted(L)
print(L)
print(L1)

In [ ]:
L=[1,34,5,7,90,2]
L1=sorted(L,reverse = True)
print(L)
print(L1)

Present sorting algorithms here primarily to provide some practice in **thinking about algorithm design and complexity analysis**

We begin with a simple but inefficient algorithm, **selection sort**

### Selection sort 

**Selection sort** works by maintaining the **loop invariant** that, given a partitioning of the list into 

* <b>a prefix (L[0:i])</b> 

* <b>a suffix (L[i+1:len(L)])</b>,

the prefix is sorted and no element in the prefix is larger than <b>the smallest element</b> in the suffix.

We use induction to reason about **loop invariants**.

* **Base case**: At the **start** of the first iteration, the **prefix** is **empty**, i.e., the
**suffix** is the **entire list**. The invariant is  true.


* **Induction step**: At each step of the algorithm, we move one element **from the suffix to the prefix.** 


>   * We do this by appending a **minimum** element of the suffix to the end of the prefix. Because the invariant held before we moved the element,


>   * we know that after we append the element the prefix is **still sorted**. 


>   * We also know that since we removed the smallest element in the suffix, **no element in the prefix is larger than the smallest element in the suffix**.


* When **the loop is exited**, the **prefix** includes the **entire list**, and the **suffix**
is **empty**.


Therefore, the entire list is now sorted in **ascending** order.

Example:
```c
int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
```
![](./img/selSort.jpg)

In [ ]:
#Page 132, Figure 10.3
def selSort(L):
    """Assumes that L is a list of elements that can be
         compared using >.
       Sorts L in ascending order"""
    suffixStart = 0
    while suffixStart != len(L):
        #look at each element in suffix
        for i in range(suffixStart, len(L)):
            if L[i] < L[suffixStart]:
                #swap position of elements
                L[suffixStart], L[i] = L[i], L[suffixStart]
        suffixStart += 1

In [ ]:
L=[1,34,5,7,90,2]
selSort(L)
print(L)

Unfortunately, it is rather inefficient

* The complexity of the inner loop is $O(len(L))$ :

```python
   for i in range(suffixStart, len(L)):
```

* The complexity of the outer loop is also $O(len(L))$ :
```python
while suffixStart != len(L):
```

The complexity of the entire function is $O(len(L)^2)$. I.e., it is **quadratic** in the length of L.

#### SelectionSort in C

In [ ]:
%%file ./code/ds/SelectionSort.c

/* Sorting an array using Selection Sort (SelectionSort.c) */

#include <stdio.h>
#include <stdlib.h> 

void selectionSort(int a[], int size);
void print(const int a[], int iMin, int iMax);

// Sort the given array of size using selection sort
void selectionSort(int a[], int size) {
   int temp; // for swaping
   for (int i = 0; i < size - 1; ++i) {
      // for tracing
      print(a, 0, i - 1);
      print(a, i, size - 1);
 
      // [0, i-1] already sort
      // Search for the smallest element in [i, size-1]
      //  and swap with a[i]
      int minIndex = i;  // assume fist element is the smallest
      for (int j = i + 1; j < size; ++j) {
         if (a[j] < a[minIndex]) minIndex = j;
      }
      if (minIndex != i) {  // swap
         temp = a[i];
         a[i] = a[minIndex];
         a[minIndex] = temp;
      }
 
      // for tracing
      printf("=> ");
      print(a, 0, i - 1);
      print(a, i, size - 1);
      printf("\n");
   }
}
 
// Print the contents of the array in [iMin, iMax]
void print(const int a[], int iMin, int iMax) {
   printf("{");
   for (int i = iMin; i <= iMax; ++i) {
      printf("%d",a[i]);
      if (i < iMax)  printf(",");
   }
   printf("}");
}
 
int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   print(a, 0, SIZE - 1);
   printf("\n"); 
   selectionSort(a, SIZE);
   print(a, 0, SIZE - 1);
   printf("\n"); 
  
   return 0;
}


In [ ]:
!gcc -o ./code/ds/SelectionSort.exe ./code/ds/SelectionSort.c

In [ ]:
!.\code\ds\SelectionSort.exe

### 10.2.1 Merge Sort

Merge sort is a prototypical <b>divide-and-conquer algorithm</b>. It was invented in 1945, by John von Neumann, and is still widely used.

In general, a divide-and-conquer algorithm is characterized by

* 1 A **threshold input size**, below which the problem is **not subdivided**

* 2 The size and number of **sub-instances** into which an instance is **split**,

* 3 The algorithm used to **combine** sub-solutions.


The threshold is sometimes called the **recursive base**. For item 2 it is usual to consider the ratio of initial problem size to sub-instance size. In most of the examples we’ve seen so far, the ratio was 2.

Like many divide-and-conquer algorithms it is most easily described recursively:

* If the list is of **length 0 or 1**, it is **already** sorted.  
 
     A **threshold input size**
     

* If the list has more than one element, **split the list** into two lists, and use **merge sort** to sort each of them.

    The size and number of **sub-instances** into which an instance is **split**,



* **Merge** the results.

   The algorithm used to **combine** sub-solutions.


<font color='blue'>**The idea**</font> is: 

>look at the **first** element of each list, and move the **smaller of the two** to the **end** of the result list.
>
>When one of the lists is empty, all that remains is to copy the remaining items from the other list.

Consider, for example, merging the two lists
```
 [1,5,12,18,19,20]
 [2,3,4,17]
```
![mergesort](./img/mergesort.png)

Example
```c
 int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
```
![](./img/mergeSort.jpg)




### What is the complexity of the merge process? 

It involves two constant-time operations： 

* 1 comparing the values of elements 

* 2 copying elements from one list to another. 

The number of comparisons is **O(len(L))**, where L is the **longer** of the two lists. 

The number of copy operations is **O(len(L1) + len(L2))**, because each element gets copied exactly once. 

Therefore, merging two sorted lists is linear in **the length of the lists**

In [ ]:
#Page 134, Figure 10.4
def merge(left, right, compare):
    """Assumes left and right are sorted lists and
         compare defines an ordering on the elements.
       Returns a new sorted (by compare) list containing the
         same elements as (left + right) would contain."""
    
    result = []
    i,j = 0, 0
    while i < len(left) and j < len(right):
        if compare(left[i], right[j]):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    while (i < len(left)):
        result.append(left[i])
        i += 1
    while (j < len(right)):
        result.append(right[j])
        j += 1
    return result

import operator
#  compare = operator.lt
def mergeSort(L, compare = operator.lt):
    """Assumes L is a list, compare defines an ordering
         on elements of L
       Returns a new sorted list containing the same elements as L"""
    if len(L) < 2:
        return L[:]
    else:
        middle = len(L)//2
        left = mergeSort(L[:middle], compare)
        right = mergeSort(L[middle:], compare)
        return merge(left, right, compare)

In [ ]:
L=[1,5,12,18,19,20,2,3,4,17]
L1=mergeSort(L,operator.gt)
print(L1)

#### Let’s analyze the complexity of mergeSort. 

We already know that the time complexity of merge is $O(len(L))$. At each level of recursion the total number of elements to be merged is $len(L)$. 

Therefore, the time complexity of mergeSort is

* $O(len(L))$ multiplied by the number of levels of recursion. 

Since mergeSort divides the list <b>in half</b> each time, we know that the number of levels of recursion is $O(log(len(L))$. 
  
Therefore, the time complexity of mergeSort is $O(n*log(n))</b>$, where n is len(L).
                                                                                                             
This improvement in time complexity comes with a price. 

<b>Selection sort</b> is an example of an <b>in-place</b> sorting algorithm. Because it works by swapping the place of elements within the list, it uses only <b>a constant amount of extra storage</b> (one element in our implementation). 

In contrast, the <b>merge sort</b> algorithm  involves making<b> copies of the list</b>. This means that its space complexity is $O(len(L))$.                                                                                                             

**A merge sort** works as follows:

* Divide the unsorted list into n sublists, each containing 1 element (a list of 1 element is considered sorted).

* Repeatedly merge sublists to produce new sorted sublists until there is only 1 sublist remaining. This will be the sorted list.

### 10.2.2 <font color="blue">Exploiting Functions</font> as Parameters

Suppose we want to sort a list of names written as firstName lastName, e.g.,

the list
```python
['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen'].
```
Figure 10.5 defines two ordering functions, and then uses these to sort a list in two different ways.
```python
def mergeSort(L, compare = operator.lt):
```

In [ ]:
#Page 135, Figure 10.5
def lastNameFirstName(name1, name2):
    name1 = name1.split(' ')
    name2 = name2.split(' ')
    if name1[1] != name2[1]:
        return name1[1] < name2[1]
    else: #last names the same, sort by first name
        return name1[0] < name2[0]

def firstNameLastName(name1, name2):
    # If that argument is omitted 
    #  the string is split using arbitrary strings of whitespace characters
    #  space, tab, newline, return, and formfeed).
    name1 = name1.split()
    name2 = name2.split()
    if name1[0] != name2[0]:
        return name1[0] < name2[0]
    else: #first names the same, sort by last name
        return name1[1] < name2[1]


In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = mergeSort(L, lastNameFirstName)
print('Sorted by last name =', newL)
newL = mergeSort(L, firstNameLastName)
print('Sorted by first name =', newL)

### MergeSort in C

In [ ]:
%%file ./code/ds/MergeSort.c

/* Sorting an array using Merge Sort (MergeSort.c) */
#include <stdio.h>
#include <stdlib.h>
 
void mSort(int a[], int size);
void mergeSort(int a[], int iLeft, int iRight, int work[]);
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]);
void print(const int a[], int iLeft, int iRight);

 
// Sort the given array of size
void mSort(int a[], int size) {
   int work[size];  // work space
   mergeSort(a, 0, size - 1, work);
}
 
// Sort the given array in [iLeft, iRight]
void mergeSort(int a[], int iLeft, int iRight, int work[]) {
   if ((iRight - iLeft) >= 1) {   // more than 1 elements, divide and sort
      // Divide into left and right half
      int iLeftHalfLeft = iLeft;
      int iLeftHalfRight = (iRight + iLeft) / 2;   // truncate
      int iRightHalfLeft = iLeftHalfRight + 1;
      int iRightHalfRight = iRight;
 
      // Recursively sort each half
      mergeSort(a, iLeftHalfLeft, iLeftHalfRight, work);
      mergeSort(a, iRightHalfLeft, iRightHalfRight, work);
 
      // Merge two halves
      merge(a, iLeftHalfLeft, iLeftHalfRight, iRightHalfLeft, iRightHalfRight, work);
   }
}
 
// Merge two halves in [iLeftHalfLeft, iLeftHalfRight] and [iRightHalfLeft, iRightHalfRight]
// Assume that iLeftHalfRight + 1 = iRightHalfLeft
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]) {
   int size = iRightHalfRight - iLeftHalfLeft + 1;
   int iResult = 0;
   int iLeft = iLeftHalfLeft;
   int iRight = iRightHalfLeft;
   while (iLeft <= iLeftHalfRight && iRight <= iRightHalfRight) {
      if (a[iLeft] <= a[iRight]) {
         work[iResult++] = a[iLeft++];
      } else {
         work[iResult++] = a[iRight++];
      }
   }
   // Copy the remaining left or right into work
   while (iLeft <= iLeftHalfRight) work[iResult++] = a[iLeft++];
   while (iRight <= iRightHalfRight) work[iResult++] = a[iRight++];
 
   // for tracing
   print(a, iLeftHalfLeft, iLeftHalfRight);
   print(a, iRightHalfLeft, iRightHalfRight);
   printf("=> ");
   print(work, 0, size - 1);
   printf("\n");
 
   // Copy the work back to the original array
   for (iResult = 0, iLeft = iLeftHalfLeft; iResult < size; ++iResult, ++iLeft) {
      a[iLeft] = work[iResult];
   }
}
 
// Print the contents of the given array from iLeft to iRight (inclusive)
void print(const int a[], int iLeft, int iRight) {
   printf("{");
   for (int i = iLeft; i <= iRight; ++i) {
      printf("%d", a[i]);
      if (i < iRight) printf(",");
   }
   printf("} ");
}

 
int main() {
   // Test 1
   const int SIZE_1 = 8;
   int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
 
   print(a1, 0, SIZE_1 - 1);
   printf("\n");
   mSort(a1, SIZE_1);
   print(a1, 0, SIZE_1 - 1);
   printf("\n \n");
 
   // Test 2
   const int SIZE_2 = 13;
   int a2[13] = {8, 4, 5, 3, 2, 9, 4, 1, 9, 1, 2, 4, 5};
 
   print(a2, 0, SIZE_2 - 1);
   printf("\n");
   mSort(a2, SIZE_2);
   print(a2, 0, SIZE_2 - 1);
   printf("\n \n");
    
   return 0;
}

In [ ]:
!gcc -o ./code/ds/MergeSort.exe ./code/ds/MergeSort.c

In [ ]:
!.\code\ds\MergeSort.exe

### The GNU C Library: Array Sort Function

http://www.gnu.org/software/libc/manual/html_node/Array-Sort-Function.html

To sort an array using an arbitrary comparison function, use the **qsort** function. The prototype for this function is in **stdlib.h**. 

Function:

```c
void qsort (void *array, size_t count, size_t size, comparison_fn_t compare)
```
To do this, you supply **a comparison function** to compare two elements of the array

This type is a GNU extension
```c
int comparison_fn_t (const void *, const void *);
```

In [ ]:
%%file ./code/ds/gccqsort.c

/* Sorting an array using The GNU C Library ：Sort (gccqsort.c) */

#include <stdio.h>
#include <stdlib.h> 

int compare_ints (const void * a, const void * b)  
{  
  return ( *(int*)a - *(int*)b );  
}  

int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   qsort(a, SIZE, sizeof(int),compare_ints);
   
   for(int i = 0; i <SIZE; i++) 
      printf("%d ", a[i]);
   
  return 0;
}
    

In [ ]:
!gcc -o ./code/ds/gccqsort.exe ./code/ds/gccqsort.c

In [ ]:
!.\code\ds\gccqsort.exe

### 10.2.3 Sorting in Python

The sorting algorithm used in most Python implementations is called 

* <b>Timsort</b> ： https://en.wikipedia.org/wiki/Timsort

The **key idea** is to take **advantage** of the fact that in a lot of data sets the data is <b>already partially sorted</b>. 

**Timsort**’s worst-case performance is the same as merge sort’s, but on average it performs considerably better.

The Python

* 1  method **list.sort** takes a list as its first argument and **modifies** that list
    
    
* 2 function **sorted** takes an iterable object (e.g., a list or a dictionary) as its first argument and returns a **new** sorted list.

**sort(*, key=None, reverse=None)**

* This method sorts the list **in place**, using only < comparisons between items. Exceptions are not suppressed - if any comparison operations fail, the entire sort operation will fail (and the list will likely be left in a partially modified state).


* list.sort() accepts two arguments that can only be passed by keyword (keyword-only arguments):


* **key** specifies a function of one argument that is used to extract a comparison key from each list element (for example, key=str.lower). The key corresponding to each item in the list is calculated once and then used for the entire sorting process. The default value of None means that list items are sorted directly without calculating a separate key value.


* **reverse** is a boolean value. If set to True, then the list elements are sorted as if each comparison were reversed.


**sorted(iterable[, key][, reverse])**

* Return a **new** sorted list from the items in iterable.


* Has two optional arguments which must be specified as keyword arguments.


* **key** specifies a function of one argument that is used to extract a comparison key from each list element: key=str.lower. The default value is None (compare the elements directly).


* **reverse** is a boolean value. If set to True, then the list elements are sorted as if each comparison were reversed.

In [ ]:
# sorted list: new list
L = [3,5,2]
print('sorted L(a new list)=',sorted(L))
print('L=',L)

# sorted dict
# dict:iterable
D = {'a':12, 'c':5, 'b':'dog'}
print('sorted D(a new list)=',sorted(D))

# list.sort in place
L.sort()
print('L(modified L)=',L)


# but :'dict' object has no attribute 'sort'
D.sort()

Both the **list.sort** method and the **sorted** function can have two additional parameters.

>The <b>key</b> parameter plays the same role as compare in our implementation of merge sort: it is used to <b>supply the comparison function</b> to be
used.


>The <b>reverse</b> parameter specifies whether the list is to be sorted in <b>ascending or descending order</b>.

In [ ]:
L = [[1,2,3], (3,2,1,0), 'abc']
print(sorted(L, key = len, reverse = True))

In [ ]:
def getlastNameFirstName(name):
    name = name.split(' ')
    return name[1]

def getfirstNameLastName(name):
    name = name.split()
    return name[0]

In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = sorted(L, key=getlastNameFirstName)
print('Sorted by last name =', newL)
newL =sorted(L, key=getfirstNameLastName)
print('Sorted by first name =', newL)

#### stable sorts

Both the **list.sort** method and the sorted function provide <b>stable sorts</b>. 

>This means that if **two elements are equal** with respect to the comparison used **in the sort** 

>their **relative ordering** in the original list (or other iterable object) is **preserved** in the final list.

## Further Reading

###  8.6. bisect — Array bisection algorithm

https://docs.python.org/3.5/library/bisect.html

This module provides support for maintaining a list in sorted order without having to sort the list after each insertion.

The module is called bisect because it uses a basic bisection algorithm to do its work.

## Further Reading

* 严蔚敏，李冬梅，吴伟民. 数据结构（C语言版），人民邮电出版社（第2版）,2015年2月  


* Mark Allen Weiss. Data Structures and Algorithm Analysis in C


* GNU C Library: Searching and Sorting : http://www.gnu.org/software/libc/manual/html_node/Searching-and-Sorting.html

